In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import sys
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tf_pose import common
import tf_slim as slim
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

In [ ]:
model='mobilenet_v2_large'
resize='480x432'
w, h = model_wh(resize)

In [ ]:
e = TfPoseEstimator(get_graph_path(model), target_size=(w, h))

In [ ]:
video_path = '/Users/andrei-macpro/Documents/Data/trial.mp4'

In [ ]:
cap = cv2.VideoCapture(video_path)


In [ ]:
def extract(human):
    """takes a human openpose object and returns parsed skeleton coordinates and probabilities """
    all_points = [x for x in range(18)]
    skeleton =[]
    probs = []
    kpts = []
    coords_x= []
    coords_y= []
    subscriptable = str(human).split("BodyPart:")[1:]
    for x in subscriptable:
        kpts.append(int((x.split('-')[0])))
        coords_x.append(float((str(x.split('-')[1]).split(" score=")[0][1:5]))*image.shape[1])
        coords_y.append(float((str(x.split('-')[1]).split(" score=")[0][6:11]))*image.shape[0])
        probs.append(float(str(x.split('-')[1]).split(" score=")[1]))
    
    for point in range(len(all_points)): # this ensures that undetected kpoints still have a value (nan) 
        if all_points[point] not in kpts:
            kpts.insert(all_points[point], point)
            coords_x.insert(all_points[point], "nan")
            coords_y.insert(all_points[point], "nan")
            probs.insert(all_points[point], 'nan')
    
    for k,cox, coy, prob in zip(kpts, coords_x, coords_y, probs): 
        if type(cox) == str:
            skeleton.append([cox,coy,prob])
        else:
            skeleton.append([round(cox,2),round(coy,2), prob])
    return(np.array(skeleton))

In [ ]:
cap = cv2.VideoCapture(video_path)
human_no = []
frame_no = 0
failed_frames=[]
folder_path = '/Users/andrei-macpro/Documents/Data/csv_files/'
while(cap.isOpened()):
    ret_val, image = cap.read()
    if ret_val == False:
        break
    frame_no+=1
    humans = e.inference(image,
                         resize_to_default=(w > 0 and h > 0),
                         upsample_size=4.0)
    human_no.append(len(humans))
    
    if len(humans)>3:
        skeleton_4 = extract(humans[3])
        skeleton_3 = extract(humans[2])
        skeleton_2 = extract(humans[1])
        skeleton_1 = extract(humans[0])
        data = pd.DataFrame((np.concatenate((skeleton_1,skeleton_2, skeleton_3, skeleton_4), axis=1)),
                            columns = ['x_1', 'y_1', 'prob_1', 'x_2', 'y_2', 'prob_2', 'x_3', 'y_3', 'prob_3', 
                                      'x_4', 'y_4', 'prob_4'])
        data.to_csv(folder_path + 'frame_' + str(frame_no) + '.csv')  
    
    
    elif len(humans)>2:
        skeleton_3 = extract(humans[2])
        skeleton_2 = extract(humans[1])
        skeleton_1 = extract(humans[0])
        data = pd.DataFrame((np.concatenate((skeleton_1,skeleton_2, skeleton_3), axis=1)),
                            columns = ['x_1', 'y_1', 'prob_1', 'x_2', 'y_2', 'prob_2', 'x_3', 'y_3', 'prob_3'])
        data.to_csv(folder_path + 'frame_' + str(frame_no) + '.csv') 
        
        
    elif len(humans)>1:
        skeleton_2 = extract(humans[1])
        skeleton_1 = extract(humans[0])
        data = pd.DataFrame((np.concatenate((skeleton_1, skeleton_2), axis=1)),
                            columns = ['x_1', 'y_1', 'prob_1', 'x_2', 'y_2', 'prob_2'])
        data.to_csv(folder_path + 'frame_' + str(frame_no) + '.csv') 
                            

    elif len(humans) == 1:
        skeleton_1 = extract(humans[0])
        data = pd.DataFrame(skeleton_1,
                            columns = ['x_1', 'y_1', 'prob_1'])
        data.to_csv(folder_path + 'frame_' + str(frame_no) + '.csv') 
                

    else:
        failed_frames.append(frame_no)
        print(frame_no)
cap.release()
    

In [ ]:
cap = cv2.VideoCapture(video_path)
kpts_final = []
coords_x_final = []
coords_y_final = []

all_points = [x for x in range(18)]
big_dic = []
original_kpts =[]
probs_final = []
while(cap.isOpened()):

    ret_val, image = cap.read()
    probs = []
    kpts = []
    coords_x= []
    coords_y= []
    if ret_val == False:
        break
    humans = e.inference(image,
                         resize_to_default=(w > 0 and h > 0),
                         upsample_size=4.0)
    print(humans[0])
    subscriptable = str(humans[0]).split("BodyPart:")[1:]

    for x in subscriptable:
        kpts.append(int((x.split('-')[0])))
        original_kpts.append(int((x.split('-')[0])))
        coords_x.append(float((str(x.split('-')[1]).split(" score=")[0][1:5]))*image.shape[1])
        coords_y.append(float((str(x.split('-')[1]).split(" score=")[0][6:11]))*image.shape[0])
        probs.append(float(str(x.split('-')[1]).split(" score=")[1]))
    
    for point in range(len(all_points)):
        if all_points[point] not in kpts:
            kpts.insert(all_points[point], point)
            coords_x.insert(all_points[point], "nan")
            coords_y.insert(all_points[point], "nan")
            probs.insert(all_points[point], 'nan')
    #kpts_final.append(kpts)
    #coords_x_final.append(coords_x)
    #coords_y_final.append(coords_y)
    #probs_final.append(probs)
    
    for k,cox, coy, prob in zip(kpts, coords_x, coords_y, probs): # when kpts is too short it will stop early 
        if type(cox) == str:
            big_dic.append({k:[cox,coy,prob]})
        else:
            big_dic.append({k:[round(cox,2),round(coy,2), prob]})



In [ ]:
path_videos = '/Users/andrei-macpro/Documents/Data/videoss/'

videos = [file for file in sorted(os.listdir(path_videos)) if file[0] !="."]
print(videos)

In [ ]:
frame_count = []
failed_frames = []
width = []
height = []
for video in tqdm(videos):
    cap = cv2.VideoCapture(path_videos+video)
    frame_no = 0
    failed_frame =[]
    os.makedirs('/Users/andrei-macpro/Documents/Data/' +video.split(".")[0], exist_ok=True)  
    folder_path = '/Users/andrei-macpro/Documents/Data/' +video.split(".")[0]+'/'
    while(cap.isOpened()):
        ret_val, image = cap.read()
        if ret_val == False:
            break
        frame_no+=1
        humans = e.inference(image,
                             resize_to_default=(w > 0 and h > 0),
                             upsample_size=4.0)


        if len(humans)==4:
            skeleton_4 = extract(humans[3])
            skeleton_3 = extract(humans[2])
            skeleton_2 = extract(humans[1])
            skeleton_1 = extract(humans[0])
            data = pd.DataFrame((np.concatenate((skeleton_1,skeleton_2, skeleton_3, skeleton_4), axis=1)),
                                columns = ['x_1', 'y_1', 'prob_1', 'x_2', 'y_2', 'prob_2', 'x_3', 'y_3', 'prob_3', 
                                          'x_4', 'y_4', 'prob_4'])
            data.to_csv(folder_path + 'frame_' + str(frame_no) + '.csv')  


        elif len(humans)==3:
            skeleton_3 = extract(humans[2])
            skeleton_2 = extract(humans[1])
            skeleton_1 = extract(humans[0])
            data = pd.DataFrame((np.concatenate((skeleton_1,skeleton_2, skeleton_3), axis=1)),
                                columns = ['x_1', 'y_1', 'prob_1', 'x_2', 'y_2', 'prob_2', 'x_3', 'y_3', 'prob_3'])
            data.to_csv(folder_path + 'frame_' +str(frame_no) + '.csv') 


        elif len(humans)==2:
            skeleton_2 = extract(humans[1])
            skeleton_1 = extract(humans[0])
            data = pd.DataFrame((np.concatenate((skeleton_1, skeleton_2), axis=1)),
                                columns = ['x_1', 'y_1', 'prob_1', 'x_2', 'y_2', 'prob_2'])
            data.to_csv(folder_path + 'frame_' +str(frame_no) + '.csv') 


        elif len(humans) == 1:
            skeleton_1 = extract(humans[0])
            data = pd.DataFrame(skeleton_1,
                                columns = ['x_1', 'y_1', 'prob_1'])
            data.to_csv(folder_path + 'frame_' + str(frame_no) + '.csv') 


        else:
            failed_frame.append(frame_no)
            print(frame_no)
            ## output failed frame file to keep consistency
            ## output video resolution
    failed_frames.append({video.split(".")[0]:failed_frame})
    frame_count.append({video.split(".")[0]:frame_no})
    width.append({video.split(".")[0]:(int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))})
    height.append({video.split(".")[0]:int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))})
    cap.release()
    


In [ ]:
from collections import defaultdict

metadata = defaultdict(list)

for w, h, count in zip(width, height, frame_count):
    for d in (w,h, count): # you can list as many input dicts as you want here
        for key, value in d.items():
            metadata[key].append(value)
    
metad = pd.DataFrame(metadata, index = ['width','height', 'frame_count']).transpose()

In [ ]:
metad.to_csv('metadata.csv')

In [ ]:
metad

In [ ]:
no_frames[4]

In [ ]:
new_dict = {}d
for key,value, ls in zip(new_dict, no_frames):
    print (ls)
    new_dict.key == ls.key()
    print (a.keys())

In [ ]:
df2= pd.DataFrame.from_dict(no_frames[1],orient='index').transpose()
df2

In [ ]:
df_final = pd.concat([df, df2], axis=1)

In [ ]:
df_final.to_csv('failed_frames.csv')

In [ ]:
failed_frames_2 =[]

for x in no_frames:
    failed_frames_2.append(pd.DataFrame.from_dict(x ,orient='index').transpose())
     
 

In [ ]:
failed_frames_2 = pd.concat([x for x in failed_frames_2], axis=1)
failed_frames_2

In [ ]:
failed_frames_2.to_csv('failed_frames_2.csv')

In [ ]:
### plot joint 0 

joint_0_x = []
joint_0_y = []
folder_path = '/Users/andrei-macpro/Documents/Data/1042_meal/'
for file in sorted_alphanumeric(os.listdir(folder_path)):
    data = pd.read_csv(folder_path+file) 
    joint_0_x.append(float(data.loc[0,['x_1']]))
    joint_0_y.append(float(data.loc[0,['y_1']]))


In [ ]:
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
plt.plot(joint_0_x, joint_0_y) 

In [ ]:
joint_0_x

In [ ]:
joint_0_y

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
from scipy import interpolate
x = sorted(joint_0_x)
y = sorted(joint_0_y)


tck = interpolate.splrep(x, y, s=0)
xnew = np.arange(0, 2*np.pi, np.pi/50)
ynew = interpolate.splev(xnew, tck, der=0)

In [ ]:
plt.figure()
plt.plot(x, y, 'x', xnew, ynew, xnew, np.sin(xnew), x, y, 'b')
plt.legend(['Linear', 'Cubic Spline', 'True'])
#plt.axis([-0.05, 6.33, -1.05, 1.05])
plt.title('Cubic-spline interpolation')
plt.show()

In [ ]:
sorted(joint_0_x)

In [ ]:
xnew